## 미래에셋 빅데이터 콘테스트
### 보험금 청구 분류
### 전처리

In [16]:
#라이브러리 불러오기
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#데이터 불러오기
train = pd.read_csv("D:/thon/data/mirae/classification/train.csv")
test = pd.read_csv("D:/thon/data/mirae/classification/test.csv")

#범주형 칼럼 선택
category_feature = [col for col in train.columns if train[col].dtypes == 'int64']
category_feature = list(set(category_feature) - set(['ID', 'target', 'base_ym']))

#범주형 칼럼 범주화
for col in category_feature:
    train[col] = train[col].astype(str)
    
for col in category_feature:
    test[col] = test[col].astype(str)
    
#train, test 더미변수 생성
train_dummy = pd.get_dummies(train, drop_first=True)
test_dummy = pd.get_dummies(test, drop_first = True)

#train 데이터에는 mdct_inu_rclss_dcd_9 칼럼이 있지만 test 데이터에는 없어서 추가해줌
test_dummy['mdct_inu_rclss_dcd_9'] = 0

In [ ]:
#csv 파일로 저장
train_dummy.to_csv('D:/thon/data/mirae/classification/train_dummy.csv')
test_dummy.to_csv('D:/thon/data/mirae/classification/test_dummy.csv')

In [19]:
#unknown 칼럼 삭제한 파일 따로 만들기
train_wo_uk = train_dummy.drop(['mdct_inu_rclss_dcd_9','prm_nvcd_99', 'inamt_nvcd_99', 'ac_ctr_diff_1', 'ac_rst_diff_1'], axis = 'columns')
test_wo_uk = test_dummy.drop(['mdct_inu_rclss_dcd_9','prm_nvcd_99', 'inamt_nvcd_99', 'ac_ctr_diff_1', 'ac_rst_diff_1'], axis = 'columns')

In [22]:
#csv 파일 만들기
train_wo_uk.to_csv('D:/thon/data/mirae/classification/train_wo_uk.csv')
test_wo_uk.to_csv('D:/thon/data/mirae/classification/test_wo_uk.csv')

### 모델 선정

In [ ]:
#라이브러리 불러오기
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler

train = pd.read_csv("D:/thon/data/mirae/classification/train.csv")
test = pd.read_csv("D:/thon/data/mirae/classification/test.csv")
train_dummy = pd.read_csv("D:/thon/data/mirae/classification/train_dummy.csv")
test_dummy = pd.read_csv("D:/thon/data/mirae/classification/test_dummy.csv")

#필요없는 항목 삭제
test_dummy.drop(['Unnamed: 0'], axis = 1, inplace = True)
test_dummy.drop(['ID'], axis = 1, inplace = True)
test_dummy.drop(['base_ym'], axis = 1, inplace = True)

train_dummy.drop(['Unnamed: 0'], axis = 1, inplace = True)
train_dummy.drop(['ID'], axis = 1, inplace = True)
train_dummy.drop(['base_ym'], axis = 1, inplace = True)

#X와 Y로 변수 구별
x_train_dummy = train_dummy.drop(['target'], axis = 1)
y_train_dummy = train_dummy['target']

#타깃 변수 문자형 변환
y_train_dummy = y_train_dummy.astype(str)

#훈련, 테스트 데이터로 나누기
x_train, x_test, y_train, y_test = train_test_split(x_train_dummy, y_train_dummy, random_state = 0)

#randomforest 적용해서 분류기 성능 확인
rf = RandomForestClassifier()
model = rf.fit(x_train, y_train)
rf.score(x_test, y_test)

#test_dummy 데이터에 모델 적용
target_pred = model.predict(test_dummy)

#예측 결과 데이터프레임화
target = pd.DataFrame(target_pred, columns = ['target'])

#결과 csv 파일 저장
target.to_csv('D:/thon/data/mirae/classification/target_predict1.csv')